## Contetnts
- 기초 연산
- 그룹 연산 : 집계, 변환, 필터링, 그룹매핑
- 함수 매핑 : map, apply, applymap
- Pivot

In [1]:
# 기초 연산

import pandas as pd

st1 = pd.Series({'국어':100,'영어':80,'수학':90})
st2 = pd.Series({'수학':80,'국어':90,'영어':80})

print(st1,'\n')
print(st2)

국어    100
영어     80
수학     90
dtype: int64 

수학    80
국어    90
영어    80
dtype: int64


In [2]:
pd.DataFrame(st1+st2)


,0
국어,190
수학,170
영어,160


In [7]:
add = st1 + st2
sub = st1 - st2
mul = st1 * st2
div = round((st1/st2),2)

# concat : 국어, 수학, 영어가 행으로 들어간다 (index)
df = pd.concat([add, sub, mul, div], axis = 1)
df.columns=['덧셈', '뺄셈', '곱셈', '나눗셈']
df

,덧셈,뺄셈,곱셈,나눗셈
국어,190,10,9000,1.11
수학,170,10,7200,1.12
영어,160,0,6400,1.00


In [10]:
# 국어, 수학, 영어가 열로 들어간다 (columns)
res = pd.DataFrame([add,sub,mul,div])
res.index = ['덧셈', '뺄셈', '곱셈', '나눗셈']
res

,국어,수학,영어
덧셈,190.00,170.00,160.0
뺄셈,10.00,10.00,0.0
곱셈,9000.00,7200.00,6400.0
나눗셈,1.11,1.12,1.0


In [12]:
import numpy as np

st1 = pd.Series({'국어':np.nan,'영어':80,'수학':90})
st2 = pd.Series({'수학':80,'국어':90})

add = st1 + st2
sub = st1 - st2
mul = st1 * st2
div = round((st1 / st2),2)

result = pd.DataFrame([add,sub,mul,div],
                     index=['덧셈','뺄셈','곱셈','나눗셈'])
result

,국어,수학,영어
덧셈,NaN,170.00,NaN
뺄셈,NaN,10.00,NaN
곱셈,NaN,7200.00,NaN
나눗셈,NaN,1.12,NaN


In [19]:
# Q. Null 값을 0으로 치환
res = result.copy()
result.fillna(0, inplace=True)
result

res['국어'].fillna(0)
res['영어'].fillna(0)
res

st1 = pd.Series({'국어':np.nan,'영어':80,'수학':90})
st2 = pd.Series({'수학':80,'국어':90})
ad = st1.add(st2, fill_value=0)
su = st1.sub(st2, fill_value=0)
mu = st1.mul(st2, fill_value=0)
di = st1.div(st2, fill_value=0)

df = pd.DataFrame([ad,su,mu,di], index=['덧셈','뺄셈','곱셈','나눗셈'])
df

,국어,수학,영어
덧셈,90.0,170.000,80.0
뺄셈,-90.0,10.000,80.0
곱셈,0.0,7200.000,0.0
나눗셈,0.0,1.125,inf


In [ ]:
# 0~11까지 float 타입으로
np.arange(12.)
# 3행 4열 2차원 배열로
np.arange(12.).reshape((3,4))

In [25]:
df1 = pd.DataFrame(np.arange(12.).reshape((3,4)), columns=list('abcd'))
df2 = pd.DataFrame(np.arange(20.).reshape((4,5)), columns=list('abcde'))

df2.loc[1, 'b'] = np.nan

df1+df2

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,NaN,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


In [29]:
# 계산 인자를 뒤집어 계산 : 원래라면 df1 - 인자값 이겠으나 r을 붙이면 반대로 인자값 - df1이된다.
df1.rdiv(1) # 1 / df1

df1.rsub(1) # 1 - df1

df1.radd(1) # 1 + df1

df1.rmul(1)

,a,b,c,d
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0


In [37]:
# 배열의 연산
arr = np.arange(12.).reshape((3,4))

# broadcasting : 전체 배열에 하나의 행 연산
# 일반 리스트에서는 적용안됨
arr - arr[0]

# DataFrame과 Series간 연산
frame = pd.DataFrame(np.arange(12.).reshape(4,3),columns=list('bde'),
                    index = ['Utah','Ohio','Texas','Oregon'])
frame - frame.iloc[0]

,b,d,e
Utah,0.0,0.0,0.0
Ohio,3.0,3.0,3.0
Texas,6.0,6.0,6.0
Oregon,9.0,9.0,9.0


In [ ]:
# 1줄인 2차원 배열
a = np.arange(6).reshape(1,6)
# 1줄인 1차원 배열
a = np.arange(6).reshape(6,) # 튜플에서 요소가 1개일 때 뒤에 ,를 써줘야한다.

a

#### 그룹 연산
* 복잡한 데이터를 어떤 기준에 따라 여러 그룹으로 나누어서 관찰할 수 있으며 이런 방식으로 분할 처리하는 것을 그룹 연산이라 함
* 그룹 연산은 데이터를 집계, 변환, 필터링하는데 효율적이며 판다스 groupby() 메소드를 사용함
* 그룹 객체 만들기(분할)
* 그룹 연산 메소드(적용-결합)

In [42]:
import pandas as pd
import seaborn as sns

# titanic 데이터셋에서 age, sex 등 5개 열을 선택하여 데이터프레임 만들기
titanic = sns.load_dataset('titanic')
df = titanic[['age', 'sex', 'class', 'fare', 'survived']]
#df = titanic.loc[:, ['age', 'sex', 'class', 'fare', 'survived']]

print("승객 수: ", len(df))
print(df.head())

# 키 피처를 뽑는데 중요한 정보를 얻을 수 있다.
print(titanic.info())

승객 수:  891
    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
1  38.0  female  First  71.2833         1
2  26.0  female  Third   7.9250         1
3  35.0  female  First  53.1000         1
4  35.0    male  Third   8.0500         0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null